<a href="https://colab.research.google.com/github/veggar/mcts/blob/master/MCTS_utc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from math import *
import random
import copy

# Do not edit follow config variables
# player const value
USER_AI = 2
USER_PLAYER = 1

STATE_VALUE_EMPTY = 0

STATE_GO = 0
STATE_DRAW = -1
STATE_END = [USER_PLAYER, USER_AI]

RESULT_VALUE_DRAW = 0.5
RESULT_VALUE_WIN = 1
RESULT_VALUE_NONE = 0
# end config variables

# game config
BOARD_SIZE = 4
MAP_SIZE = BOARD_SIZE*BOARD_SIZE
CHECK_END_CASE_POSITION = [
    (0,1,2), (3,4,5), (6,7,8),
    (0,3,6), (1,4,7), (2,5,8),
    (0,4,8), (2,4,6)
]

if BOARD_SIZE == 4:
    CHECK_END_CASE_POSITION = [
        (0,1,2,3), (4,5,6,7), (8,9,10,11), (12,13,14,15),
        (12,9,6,3),	(0,4,8,12),	(1,5,9,13),	(2,6,10,14),
        (3,7,11,15), (0,5,10,15)
    ]

vsAI = True

# Value for MCTS
C = 2


def getNextTurnUser(user):
    if user == USER_PLAYER:
        return USER_AI
    return USER_PLAYER 

class Game:
    def __init__(self, startUser=USER_AI):
        self.state = []
        for i in range(MAP_SIZE):
            self.state.append(STATE_VALUE_EMPTY)

        self.userToPlay = startUser

    def getMovablePosition(self):
        if self.checkEnd() != STATE_GO:
            return []
        else:
            moves = []
            for i in range(len(self.state)):
                if self.state[i] == STATE_VALUE_EMPTY:
                    moves.append(i)
                    
            return moves

    def getResult(self, player):
        result = self.checkEnd()

        if result == STATE_DRAW:
            return RESULT_VALUE_DRAW
        
        elif result == player:
            return RESULT_VALUE_WIN
        else:
            return RESULT_VALUE_NONE

    def checkEnd(self):
        
        # result = -5 # for check process for using tuple

        # slow than fixed board size
        for tup in CHECK_END_CASE_POSITION:
            first_value = self.state[tup[0]]
            all_same = True
            for pos in tup[1:]:
                all_same = all_same and first_value == self.state[pos]
                
            if all_same is True and first_value != STATE_VALUE_EMPTY:
                return first_value

        # for (x,y,z) in CHECK_END_CASE_POSITION:
        #     if self.state[x] == self.state[y] == self.state[z]:
        #         if self.state[x] != STATE_VALUE_EMPTY:
        #             # assert result == self.state[x]
        #             return self.state[x]
                
        if STATE_VALUE_EMPTY not in self.state:
            return STATE_DRAW

        return STATE_GO

    # input
    #   self.userToPlay: just before play user
    #   moveCase: moving position for next play user
    # return
    #   play or not
    def doMove(self, movePosition): 
        if len(self.state) > movePosition and movePosition >= 0 and self.state[movePosition] == STATE_VALUE_EMPTY:
            self.userToPlay = getNextTurnUser(self.userToPlay)
            self.state[movePosition] = self.userToPlay
            return True
        return False
            
    def copy(self):
        game = Game()
        game.state = self.state[:]
        game.userToPlay = self.userToPlay
        return game

    def __repr__(self):
        s = ""
        for i in range(len(self.state)):
            s += ".0X"[self.state[i]]+" "
            if i % BOARD_SIZE == BOARD_SIZE-1:
                s += "\n"
        return s

class Node(object):
    def __init__(self, movePosition=0, parent=None, game=None):
        self.parent = parent
        self.movePosition = movePosition      # case of moving
        self.wins = 0       # winning count after ith moving
        self.visits = 0    # simulation count after ith moving
        self.child = []
        self.game = game
    
        if game != None:
            self.untriedMovePositions = game.getMovablePosition()
            self.userToPlay = game.userToPlay
        else:
            self.untriedMovePositions = []
            self.userToPlay = None

    # ascending
    def selectChild(self):
        s = sorted(self.child, key = lambda c: c.wins/c.visits + sqrt(C * log(self.visits) / c.visits))
        return s[-1]

    def addChild(self, movePosition ,game):
        node = Node(movePosition = movePosition, parent = self, game = copy.deepcopy(game))
        self.untriedMovePositions.remove(movePosition)
        self.child.append(node)
        return node

    def update(self, win):
        self.visits += 1
        self.wins += win
        
    def __repr__(self):
        return "[M {0:2.0f}".format(self.movePosition+1) + " W/V {0:5.0f} /{1:5.0f} = {2:0.5f}".format(self.wins, self.visits, self.wins/self.visits)+" U" + str(self.untriedMovePositions) + "]"

    def childToString(self):
        s =""
        for c in self.child:
            s += str(c) + "\n"
        return s

def UCT(game, itermax):
    rootnode = Node(game = game)

    if vsAI is True:
        print ("{0}'s turn".format(["AI2", "AI"][game.userToPlay-1]))
    else:
        print ("{0}'s turn".format(["PLAYER", "AI"][game.userToPlay-1]))

    for i in range(itermax):
        node = rootnode
        state = copy.deepcopy(game)
        
        #selection
        while len(node.untriedMovePositions) == 0 and len(node.child) != 0:
            node = node.selectChild()
            res = state.doMove(node.movePosition)
            assert res
        
        #Expansion
        if len(node.untriedMovePositions) != 0:
            m = random.choice(node.untriedMovePositions)
            state.doMove(m)
            node = node.addChild(m, state)
        
        #simulation
        while len(state.getMovablePosition()) != 0:
            state.doMove(random.choice(state.getMovablePosition()))
        
        #BackPropagation
        while node != None:
            node.update(state.getResult(node.userToPlay))
            node = node.parent
                
    s = sorted(rootnode.child, key = lambda c: c.wins/c.visits) # ?
    res = sorted(s, key = lambda c: c.visits)
    for i in range(len(res)-1, 0, -1):
        print (str(res[i]))
    return res[-1].movePosition
    
        
def UCTPlayGame(startUser):
    game = Game(startUser=startUser)
    while len(game.getMovablePosition()) != 0:
        print (str(game))
        if game.userToPlay == USER_AI:
            rootstate = copy.deepcopy(game)
            m = UCT(rootstate, itermax = 10000)
            print ("AI best move: " + str(m+1) + "\n")
        else:
            if vsAI is True:
                rootstate2 = copy.deepcopy(game)
                m = UCT(rootstate2, itermax = 100)
                print ("AI2 best move: " + str(m+1) + "\n")
            else:
                m = input("which Do you want? {0}: ".format([i+1 for i in game.getMovablePosition()]))
                m = int(m) - 1
                print ("User Move: " + str(m+1) + "\n")
        
        game.doMove(m)
        
    print (str(game)) # display end state

    if game.getResult(game.userToPlay) == RESULT_VALUE_WIN:
        user = "AI"
        if game.userToPlay == USER_AI:
            user = "User"

        print (""+user+" Player Wins!!")
 
    else: print ("Draw!!")
    
import time


if __name__ == "__main__":    
    start_time = time.time()
    UCTPlayGame(USER_PLAYER)
    print("--- %s seconds ---" % (time.time() - start_time))


    




. . . . 
. . . . 
. . . . 
. . . . 

AI2's turn
[M  7 W/V    10 /   11 = 0.86364 U[5, 9, 11, 12, 13]]
[M  2 W/V     7 /    9 = 0.77778 U[0, 3, 4, 5, 6, 8, 14]]
[M  9 W/V     5 /    8 = 0.62500 U[0, 4, 5, 6, 9, 11, 12, 14]]
[M 12 W/V     4 /    7 = 0.57143 U[1, 2, 3, 5, 9, 10, 12, 14, 15]]
[M  8 W/V     3 /    6 = 0.50000 U[0, 2, 4, 8, 10, 11, 12, 13, 14, 15]]
[M  4 W/V     3 /    6 = 0.50000 U[0, 1, 4, 7, 8, 9, 11, 13, 14, 15]]
[M  6 W/V     3 /    6 = 0.50000 U[0, 2, 3, 4, 6, 7, 9, 10, 14, 15]]
[M 10 W/V     3 /    6 = 0.50000 U[0, 2, 3, 4, 5, 8, 10, 12, 13, 15]]
[M 16 W/V     2 /    6 = 0.41667 U[0, 3, 4, 5, 7, 8, 9, 10, 11, 14]]
[M 11 W/V     2 /    6 = 0.41667 U[0, 2, 3, 4, 5, 6, 9, 12, 14, 15]]
[M 13 W/V     2 /    5 = 0.40000 U[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 14]]
[M  5 W/V     2 /    5 = 0.40000 U[0, 1, 2, 5, 8, 9, 10, 12, 13, 14, 15]]
[M  1 W/V     2 /    5 = 0.40000 U[1, 3, 4, 5, 7, 8, 9, 11, 12, 13, 14]]
[M 14 W/V     2 /    5 = 0.40000 U[1, 4, 5, 6, 7, 8, 10, 11, 12, 14, 15]]